In [ ]:
is_startup = True

In [ ]:
if is_startup:
  !pip install transformers[torch]
  !pip install accelerate -U

In [ ]:
from google.colab import drive
import sys
drive.mount('/content/drive')
project_path = "/content/drive/MyDrive/Detalytics/chatbot/"
sys.path.insert(0, project_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
if is_startup:
  os.chdir(project_path)
  print (os.getcwd())
  !pip install -e .
  !pip install -e alutils

/content/drive/MyDrive/Detalytics/chatbot
Obtaining file:///content/drive/MyDrive/Detalytics/chatbot
  Preparing metadata (setup.py) ... done
  Attempting uninstall: chatbot
    Found existing installation: chatbot 0.0.0
    Uninstalling chatbot-0.0.0:
      Successfully uninstalled chatbot-0.0.0
  Running setup.py develop for chatbot
Obtaining file:///content/drive/MyDrive/Detalytics/chatbot/alutils
  Preparing metadata (setup.py) ... done
  Attempting uninstall: alutils
    Found existing installation: alutils 0.0.0
    Uninstalling alutils-0.0.0:
      Successfully uninstalled alutils-0.0.0
  Running setup.py develop for alutils


In [ ]:
from torch.utils.data import Dataset
import json

class ChatData(Dataset):
    def __init__(self, path:str, tokenizer):
        self.data = json.load(open(path, "r"))

        self.X = []
        for dialog in self.data:
            for turn in dialog['dialog']:
                self.X.append(turn['text'])

        for id, turn in enumerate(self.X[:-1]):
            self.X[id] = "<startofstring> " + turn +" <bot>: "+self.X[id+1] + " <endofstring>"

        # self.X = self.X[:5000]
        for one_turn_dialog in self.X[:10]:
            print(one_turn_dialog)

        self.X_encoded = tokenizer(self.X, max_length=64, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch

In [ ]:
from chatbot import constants

def train(chatData, model, optim, num_epochs):
    for num_epoch in tqdm.tqdm(range(num_epochs)):
        avg_loss = []
        for X, a in chatData:
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            avg_loss.append(loss.item())
            loss.backward()
            optim.step()
        avg_loss = sum(avg_loss) / len(avg_loss)
        print (f"Epoch {num_epoch+1}: loss = {avg_loss:.4f}")
        torch.save(model.state_dict(), f"{constants.CKPT_PATH}programming_hut/model_state.pt")
        print(infer("hello how are you"))

def infer(prompt):
    prompt = f"<startofstring> {prompt} <bot>: "
    input = tokenizer(prompt, return_tensors="pt")
    X = input["input_ids"].to(device)
    a = input["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a )
    output = tokenizer.decode(output[0])
    return output

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>",
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])

1

In [ ]:
num_epochs = 4096
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model = model.to(device)

# print(tokenizer.decode(model.generate(**tokenizer("hey i was good at basketball but ",
#                          return_tensors="pt"))[0]))

chatData = ChatData(f"{constants.DATA_PATH}convai_chat_data.json", tokenizer)
chatData =  DataLoader(chatData, batch_size=32)

model.train()

optim = Adam(model.parameters(), lr=1e-4)

print("training .... ")
train(chatData, model, optim, num_epochs)

<startofstring> I love iphone! i just bought new iphone! <bot>: Thats good for you, i'm not very into new tech <endofstring>
<startofstring> Thats good for you, i'm not very into new tech <bot>: I am a college student and i am a college student <endofstring>
<startofstring> I am a college student and i am a college student <bot>: I am go to gym and live on donations <endofstring>
<startofstring> I am go to gym and live on donations <bot>: I am a vegan and i am in the midwest <endofstring>
<startofstring> I am a vegan and i am in the midwest <bot>: So vegan... i have dogs maybe i should told then that they may eat cheap salads insted of meat <endofstring>
<startofstring> So vegan... i have dogs maybe i should told then that they may eat cheap salads insted of meat <bot>: I would not mind having them in the office that would be hard for me <endofstring>
<startofstring> I would not mind having them in the office that would be hard for me <bot>: Dogs or vegan in office? <endofstring>
<star

  0%|          | 0/4096 [00:00<?, ?it/s]

Epoch 1: loss = 2.5541


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  0%|          | 1/4096 [00:20<23:33:41, 20.71s/it]

<startofstring> hello how are you  <bot>:   <bot>: <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
Epoch 2: loss = 1.0818


...

<startofstring> hello how are you  <bot>:  <endofstring><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
Epoch 2259: loss = 0.1310


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 55%|█████▌    | 2259/4096 [13:01:18<10:33:23, 20.69s/it]

<startofstring> hello how are you  <bot>:  <endofstring><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
Epoch 2260: loss = 0.1310


 55%|█████▌    | 2259/4096 [13:01:38<10:35:37, 20.76s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 619, in save
    _save(obj, opened_zipfile, pickle_module, pickle_protocol, _disable_byteorder_record)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 853, in _save
    zip_file.write_record(name, storage.data_ptr(), num_bytes)
RuntimeError: [enforce fail at inline_container.cc:588] . PytorchStreamWriter failed writing file data/142: file write failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-f76c068c9641>", line 19, in <cell line: 19>
    train(chatData, model, optim, num_epochs)
  File "<ipython-input-15-7bc0e07da2ad>", line 16, in train
    torch.save(model.state_dict(), f"{constants.CKPT_PATH}prog